In [193]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import pandas as pd

In [194]:
data = r"D:\Big_Data\\food_price.csv"
collected_data = pd.read_csv(data)
collected_data = collected_data[2:]

collected_data["Canada"] = collected_data["Canada"].astype(float)
collected_data["Geography"] = collected_data["Geography"].str.replace(r'\s+\d+$', '', regex=True)
collected_data = collected_data.reset_index(drop=True)
display(collected_data)


,Geography,Canada
0,"Beef stewing cuts, per kilogram",16.68
1,"Beef striploin cuts, per kilogram",27.59
2,"Beef top sirloin cuts, per kilogram",18.57
3,"Beef rib cuts, per kilogram",23.80
4,"Ground beef, per kilogram",11.72
...,...,...
101,"Pasta sauce, 650 millilitres",2.92
102,"Salad dressing, 475 millilitres",3.36
103,"Almonds, 200 grams",4.35
104,"Peanuts, 450 grams",4.02


In [195]:
collected_data["Geography"] = collected_data["Geography"].astype(str)
geo_df = collected_data["Geography"].str.split(",", expand=True)
geo_df.columns = ['col1', 'col2']
collected_data["Geography"] = geo_df["col1"].apply(lambda x: x.lower())
collected_data["Units"] = geo_df["col2"]

display(geo_df)
display(collected_data)

,col1,col2
0,Beef stewing cuts,per kilogram
1,Beef striploin cuts,per kilogram
2,Beef top sirloin cuts,per kilogram
3,Beef rib cuts,per kilogram
4,Ground beef,per kilogram
...,...,...
101,Pasta sauce,650 millilitres
102,Salad dressing,475 millilitres
103,Almonds,200 grams
104,Peanuts,450 grams


,Geography,Canada,Units
0,beef stewing cuts,16.68,per kilogram
1,beef striploin cuts,27.59,per kilogram
2,beef top sirloin cuts,18.57,per kilogram
3,beef rib cuts,23.80,per kilogram
4,ground beef,11.72,per kilogram
...,...,...,...
101,pasta sauce,2.92,650 millilitres
102,salad dressing,3.36,475 millilitres
103,almonds,4.35,200 grams
104,peanuts,4.02,450 grams


In [196]:
collected_data = collected_data.rename(columns={"Canada": "Price"})


def try_convert_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return None

collected_data["Units"] = collected_data["Units"].str.strip()
collected_data["Unit_weight"] = collected_data["Units"].str.split(" ")
collected_data["Unit_weight"] = collected_data["Unit_weight"].apply(lambda x: x[0] if x else None)
collected_data["Unit_weight"] = collected_data["Unit_weight"].apply(lambda x: try_convert_float(x))


collected_data.iloc[79]


Geography      roasted or ground coffee
Price                              6.37
Units                         340 grams
Unit_weight                       340.0
Name: 79, dtype: object

In [197]:
collected_data["Price"] = collected_data.apply(
    lambda row: row["Price"] / row["Unit_weight"] if pd.notna(row["Unit_weight"]) else row["Price"], axis=1)
collected_data

,Geography,Price,Units,Unit_weight
0,beef stewing cuts,16.680000,per kilogram,NaN
1,beef striploin cuts,27.590000,per kilogram,NaN
2,beef top sirloin cuts,18.570000,per kilogram,NaN
3,beef rib cuts,23.800000,per kilogram,NaN
4,ground beef,11.720000,per kilogram,NaN
...,...,...,...,...
101,pasta sauce,0.004492,650 millilitres,650.0
102,salad dressing,0.007074,475 millilitres,475.0
103,almonds,0.021750,200 grams,200.0
104,peanuts,0.008933,450 grams,450.0


In [198]:
collected_data = collected_data.drop(columns="Unit_weight")
collected_data['Units'] = collected_data['Units'].str.replace(r'\d+', '', regex=True)  # Removes digits
collected_data['Units'] = collected_data['Units'].str.replace('per ', '')  # Removes the prefix 'per'
collected_data['Units'] = collected_data['Units'].str.strip()  # Removes leading/trailing spaces
collected_data['Units'] = collected_data['Units'].str.strip(".")  # Removes leading/trailing spaces
collected_data['Units'] = collected_data['Units'].str.strip() 
collected_data


,Geography,Price,Units
0,beef stewing cuts,16.680000,kilogram
1,beef striploin cuts,27.590000,kilogram
2,beef top sirloin cuts,18.570000,kilogram
3,beef rib cuts,23.800000,kilogram
4,ground beef,11.720000,kilogram
...,...,...,...
101,pasta sauce,0.004492,millilitres
102,salad dressing,0.007074,millilitres
103,almonds,0.021750,grams
104,peanuts,0.008933,grams


In [199]:
collected_data.to_csv("cleaned_food_data.csv")